# Segmenting and Clustering Neighborhoods in Toronto

In [81]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

### Use BeautifulSoup api to get the table from the WikiPedia page


In [9]:
class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        tb = soup.find('table', class_='wikitable sortable')
        return self.parse_html_table(tb)


    def parse_html_table(self, table):
        n_columns = 0
        n_rows = 0
        column_names = []
        for row in table.find_all('tr'):

            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows += 1
                if n_columns == 0:

                    n_columns = len(td_tags)


            th_tags = row.find_all('th')
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text().strip("\n"))

        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0, n_columns)
        df = pd.DataFrame(columns=columns,
                          index=range(0, n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker, column_marker] = column.get_text().strip('\n')
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        return df


In [19]:
hp = HTMLTableParser()
tableDF = hp.parse_url("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
print(tableDF.head())
print(tableDF.shape)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(289, 3)


### Remove rows where Borough & Neighbourhood are Not assigned

In [23]:
tableDF = tableDF[tableDF.Borough != 'Not assigned']

In [24]:
print('There are %d unique Postal Codes in the table' % tableDF.Postcode.nunique())
print('There are %d unique Boroughs in the table' % tableDF.Borough.nunique())
print('There are %d unique Neighbourhoods in the table' % tableDF.Neighbourhood.nunique())

There are 103 unique Postal Codes in the table
There are 11 unique Boroughs in the table
There are 210 unique Neighbourhoods in the table


### combined into one row with the neighborhoods separated with a comma

In [27]:
result = pd.DataFrame(tableDF.groupby(
    ['Postcode', 'Borough'])['Neighbourhood'].apply(
    lambda x: ', '.join(x))).reset_index()
result.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [42]:
result.loc[result['Neighbourhood']=='Not assigned','Neighbourhood'] = result['Borough']
result.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### rename coulmn from PostCode to Postalcode

In [43]:
result = result.rename(columns = {"Postcode": "PostalCode"})
result.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
result.shape

(103, 3)

Not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Using csv file to create the following dataframe:

In [52]:
geoDF = pd.read_csv("D:\\Softwares\\Geospatial_Coordinates.csv")
geoDF = geoDF.rename(columns = {"Postal Code": "PostalCode"})
geoDF.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, with the latitude and the longitude coordinates of each neighborhood.

In [53]:
joinDF = pd.merge(result,geoDF, how='left');
joinDF.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Segmenting and Clustering Neighborhoods in Toronto: Part 3

In [55]:
import folium 

# Library to handle JSON files
import json 

# Import Nominatim
# Converts an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Library to handle requests
import requests 

# Tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 
# Pandas library for data analsysis

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

### Define Foursquare Credentials and Version
CLIENT_ID = 'ZEROGCMHUQU2NFT2HZ0OQEAQIJESOS5KIIQ2HMSPZ15OQQCG'
CLIENT_SECRET = 'W35H51KPQ5UP4O2N0CXPBAHQ4OR2OLJLL5XS23DF4MPDWTN0'
VERSION = '20180605'

# Limit of number of venues returned by Foursquare API
LIMIT = 100 
# Define radius of search for Foursquare API
radius = 1000 

### Helper functions used while exploring the neighborhoods in Toronto

In [56]:
# xtracts the category of the venue
def category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
# Function to get Nearby Venues
def getVenuesNearBy(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
# Function to sort the venues in descending order.
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

### calling getNearByVenues() function to fetch the venues using foursquare api

In [67]:
toronto_venues = getVenuesNearBy(names=joinDF['PostalCode'],
                                 latitudes=joinDF['Latitude'],
                                 longitudes=joinDF['Longitude']
                                )

In [68]:
print(toronto_venues.shape)
toronto_venues.head()

(2228, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [70]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,1,1,1,1,1,1
M1C,1,1,1,1,1,1
M1E,7,7,7,7,7,7
M1G,3,3,3,3,3,3
M1H,7,7,7,7,7,7
M1J,1,1,1,1,1,1
M1K,7,7,7,7,7,7
M1L,9,9,9,9,9,9
M1M,3,3,3,3,3,3


Filter out / Excluding Postal codes with fewer than 10 venues

In [72]:
size = toronto_venues.Neighborhood.value_counts()
idx = size[size >= 10].index
toronto_venues_filtered = toronto_venues[toronto_venues.Neighborhood.isin(idx)]
toronto_venues_filtered.shape

(2008, 7)

In [74]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_filtered[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Move Neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns= cols)

In [76]:
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
60,M1T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [79]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1T,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.090909,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.

In [85]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(47, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1T,Pizza Place,Fast Food Restaurant,Thai Restaurant,Noodle House,Chinese Restaurant,Rental Car Location,Shopping Mall,Italian Restaurant,Fried Chicken Joint,Pharmacy
1,M1W,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Noodle House,Sandwich Place,Breakfast Spot,Electronics Store,Pharmacy,Grocery Store
2,M2J,Clothing Store,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Cosmetics Shop,Food Court,Women's Store,Bakery,Boutique,Sandwich Place
3,M2N,Ramen Restaurant,Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Café,Lounge,Ice Cream Shop,Sushi Restaurant,Indonesian Restaurant
4,M3C,Asian Restaurant,Gym,Grocery Store,Beer Store,Coffee Shop,Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Discount Store
5,M3H,Coffee Shop,Deli / Bodega,Shopping Mall,Fast Food Restaurant,Bridal Shop,Sandwich Place,Diner,Supermarket,Bank,Sushi Restaurant
6,M4B,Fast Food Restaurant,Pizza Place,Gastropub,Bank,Athletics & Sports,Rock Climbing Spot,Pharmacy,Intersection,Pet Store,Gym / Fitness Center
7,M4G,Sporting Goods Shop,Coffee Shop,Sushi Restaurant,Burger Joint,Sports Bar,Beer Store,Bike Shop,Smoothie Shop,Shopping Mall,Breakfast Spot
8,M4H,Indian Restaurant,Yoga Studio,Bank,Grocery Store,Housing Development,Liquor Store,Discount Store,Park,Pharmacy,Pizza Place
9,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant


Cluster Neighborhoods Run k-means to cluster the neighborhood into 5 clusters.

In [87]:
# Set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 2, 0, 2, 3, 4, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [98]:
toronto_merged = joinDF[joinDF.PostalCode.isin(idx)].copy()
toronto_merged.rename(columns={'PostalCode': 'Neighborhood'}, inplace=True)

# Add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
toronto_merged

(47, 16)


,Neighborhood,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,3,Pizza Place,Fast Food Restaurant,Thai Restaurant,Noodle House,Chinese Restaurant,Rental Car Location,Shopping Mall,Italian Restaurant,Fried Chicken Joint,Pharmacy
15,M1W,Scarborough,"L'Amoreaux West, Steeles West",43.799525,-79.318389,3,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Noodle House,Sandwich Place,Breakfast Spot,Electronics Store,Pharmacy,Grocery Store
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2,Clothing Store,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Cosmetics Shop,Food Court,Women's Store,Bakery,Boutique,Sandwich Place
22,M2N,North York,Willowdale South,43.770120,-79.408493,0,Ramen Restaurant,Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Café,Lounge,Ice Cream Shop,Sushi Restaurant,Indonesian Restaurant
27,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,2,Asian Restaurant,Gym,Grocery Store,Beer Store,Coffee Shop,Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Discount Store
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,3,Coffee Shop,Deli / Bodega,Shopping Mall,Fast Food Restaurant,Bridal Shop,Sandwich Place,Diner,Supermarket,Bank,Sushi Restaurant
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,4,Fast Food Restaurant,Pizza Place,Gastropub,Bank,Athletics & Sports,Rock Climbing Spot,Pharmacy,Intersection,Pet Store,Gym / Fitness Center
38,M4G,East York,Leaside,43.709060,-79.363452,0,Sporting Goods Shop,Coffee Shop,Sushi Restaurant,Burger Joint,Sports Bar,Beer Store,Bike Shop,Smoothie Shop,Shopping Mall,Breakfast Spot
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372,0,Indian Restaurant,Yoga Studio,Bank,Grocery Store,Housing Development,Liquor Store,Discount Store,Park,Pharmacy,Pizza Place
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant


In [124]:
# create map
%matplotlib inline  
toronto_latitude = 43.70011
toronto_longitude = -79.4163
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)


map_clusters.save('toronto_map.html')
print(map_clusters)
from IPython.display import HTML
HTML('<iframe src=toronto_map.html width=700 height=450></iframe>')

### 5. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.


### Cluster 1

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,0,Ramen Restaurant,Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Café,Lounge,Ice Cream Shop,Sushi Restaurant,Indonesian Restaurant
38,East York,0,Sporting Goods Shop,Coffee Shop,Sushi Restaurant,Burger Joint,Sports Bar,Beer Store,Bike Shop,Smoothie Shop,Shopping Mall,Breakfast Spot
39,East York,0,Indian Restaurant,Yoga Studio,Bank,Grocery Store,Housing Development,Liquor Store,Discount Store,Park,Pharmacy,Pizza Place
41,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
42,East Toronto,0,Board Shop,Italian Restaurant,Pet Store,Gym,Coffee Shop,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
43,East Toronto,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant
47,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Thai Restaurant,Café,Sushi Restaurant,Coffee Shop,Deli / Bodega,Chinese Restaurant
49,Central Toronto,0,Coffee Shop,Convenience Store,Pub,Medical Center,Bagel Shop,Pizza Place,Light Rail Station,Vietnamese Restaurant,Fried Chicken Joint,Supermarket
51,Downtown Toronto,0,Coffee Shop,Restaurant,Park,Bakery,Pizza Place,Pub,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Pub,Mediterranean Restaurant,Café,Men's Store


### cluster 2

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Sculpture Garden


### Cluster 3

In [130]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,North York,2,Clothing Store,Fast Food Restaurant,Coffee Shop,Asian Restaurant,Cosmetics Shop,Food Court,Women's Store,Bakery,Boutique,Sandwich Place
27,North York,2,Asian Restaurant,Gym,Grocery Store,Beer Store,Coffee Shop,Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Discount Store
46,Central Toronto,2,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Cosmetics Shop,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Park
71,North York,2,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Sporting Goods Shop,Event Space,Miscellaneous Shop,Athletics & Sports,Vietnamese Restaurant,Coffee Shop
92,Etobicoke,2,Social Club,Sandwich Place,Discount Store,Burrito Place,Supplement Shop,Burger Joint,Flower Shop,Bakery,Convenience Store,Grocery Store


### Cluster 4

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,3,Pizza Place,Fast Food Restaurant,Thai Restaurant,Noodle House,Chinese Restaurant,Rental Car Location,Shopping Mall,Italian Restaurant,Fried Chicken Joint,Pharmacy
15,Scarborough,3,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Noodle House,Sandwich Place,Breakfast Spot,Electronics Store,Pharmacy,Grocery Store
28,North York,3,Coffee Shop,Deli / Bodega,Shopping Mall,Fast Food Restaurant,Bridal Shop,Sandwich Place,Diner,Supermarket,Bank,Sushi Restaurant
101,Etobicoke,3,Grocery Store,Pharmacy,Japanese Restaurant,Pizza Place,Liquor Store,Beer Store,Discount Store,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant


### Clustrer 5

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York,4,Fast Food Restaurant,Pizza Place,Gastropub,Bank,Athletics & Sports,Rock Climbing Spot,Pharmacy,Intersection,Pet Store,Gym / Fitness Center
